Group Memebers

*   Chirag Gupta - 10623123
*   Chukwuebuka Uwaoma - 10621864
*   Dipanshu



In [16]:
# Import necesarry libraries
import tweepy
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import mysql.connector
from datetime import datetime
import re

In [17]:
# Load dotenv and read secret keys and tokens
load_dotenv()
api_key = os.getenv('api_key')
api_secret_key = os.getenv('api_secret_key')
access_token = os.getenv('access_token')
access_token_secret = os.getenv('access_token_secret')

In [18]:
# Connect to twitter using tweepy API and stored tokens
auth = tweepy.OAuthHandler(api_key,api_secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [262]:
# Pulls tweets based on search query and geocode location
tweetlist = []
geocode="53.1424,-7.6921,300km" # "53.3498053,-6.2603097,25km"
filter =  '-filter:retweets AND -filter:replies'
query = 'housing'
tweets = tweepy.Cursor(api.search_tweets, q=query+filter, geocode=geocode, lang='en', tweet_mode="extended", count=20).items(20)

# adds tweets to a dataframe
column_names = ["id", "created_at", "username", "tweet", "retweet_count", "favorite_count", "user_location"]
df = pd.DataFrame(columns=column_names)

for tweet in tweets:
    #create a new row object
    new_row = pd.Series({
        "id": tweet.id, 
        "created_at":tweet.created_at,
        "username": tweet.user.screen_name, 
        "tweet": tweet.full_text, 
        "retweet_count": tweet.retweet_count, 
        "favorite_count": tweet.favorite_count,
        "user_location":tweet.user.location,
    })
    
    # get the index of the last row
    last_row_index = len(df)
    # add the new row to the DataFrame using .iloc[]
    df.loc[last_row_index] = new_row
df.head()
    
# Pull tweet text, id, created_at_date,  username, retweets, likes saved to temporary dataframe - Virgil - DONE

,id,created_at,username,tweet,retweet_count,favorite_count,user_location
0,1646979425718747136,2023-04-14 20:51:09+00:00,DeptHousingIRL,We have schemes like this commencing on sites ...,0,0,"Dublin, Ireland"
1,1646968527264403472,2023-04-14 20:07:50+00:00,Robcass78,Wow. Ought to be front page of Sunday business...,0,0,"Dungarvan, Waterford, Ireland"
2,1646962787657175056,2023-04-14 19:45:02+00:00,ConorKellyPBP,WE DO NOT NEED FF or FG! \n\n🏠 We need a rent ...,0,2,Dublin
3,1646959435657498747,2023-04-14 19:31:43+00:00,fzoley,That second pic omg they’re in their housing e...,0,2,Doire
4,1646959214311424007,2023-04-14 19:30:50+00:00,3ddesignbureau,Check out one of our latest works! A new 15M #...,0,0,Dublin Ireland


In [20]:
# Clean the data
    ## clean text 
    ## date_time in right format eg dd/mm/yyyy - Chirag

# connect to a external DB upload cleaned data
    ## make sure tweets are unique by id - Dipanshu

# pull from database assign to dataframe - DP 
    # tokenize - virgil
    # lematize  - Dipanshu
    # visualise word cloud - Chirag

In [263]:
# Pre-processing the data

# Formatting the date to dd/mm/YYYY format
def formatDate(timestamp):
    # As the timestamp retrieved from the api is already in DatetimeIndex form, simply running string format function 
    # of the datetime library and passing the desired format
    formatted_date = timestamp.strftime("%d/%m/%Y")
    return formatted_date

# Getting only the words and numbers from the tweet that contains links, special symbols, emojis, etc.
def cleanTweet(tweet):
    # Converting the string to lowercase and replacing the ' with blank
    removed_apostrophe = tweet.lower().replace("'","")
    # Using regex pattern to replace the links, <html> tags and &amp; symbol (seen frequently in the tweets) with blank
    removed_links_tags_frequentsymbol = re.sub("(https?://\S+)|(<[\w]+>)|(&amp;)","",removed_apostrophe)
    # This regex pattern returns a list that contains all the words and numbers in the tweet
    cleaned_tweet = re.findall("([a-z0-9]+)", removed_links_tags_frequentsymbol)
    return cleaned_tweet

# Applying respective functions to both columns in the dataset
df["created_at"] = df["created_at"].apply(lambda x: formatDate(x))
df["tweet"] = df["tweet"].apply(lambda x: cleanTweet(x))
df.head()

,id,created_at,username,tweet,retweet_count,favorite_count,user_location
0,1646979425718747136,14/04/2023,DeptHousingIRL,"[we, have, schemes, like, this, commencing, on...",0,0,"Dublin, Ireland"
1,1646968527264403472,14/04/2023,Robcass78,"[wow, ought, to, be, front, page, of, sunday, ...",0,0,"Dungarvan, Waterford, Ireland"
2,1646962787657175056,14/04/2023,ConorKellyPBP,"[we, do, not, need, ff, or, fg, we, need, a, r...",0,2,Dublin
3,1646959435657498747,14/04/2023,fzoley,"[that, second, pic, omg, they, re, in, their, ...",0,2,Doire
4,1646959214311424007,14/04/2023,3ddesignbureau,"[check, out, one, of, our, latest, works, a, n...",0,0,Dublin Ireland


In [3]:
connection = mysql.connector.connect(host='localhost',
                                         database='ca',
                                         user='root',
                                         password='toor')

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (10061)

In [16]:
mySql_insert_query = 'CREATE TABLE if not exists Perso (PersonID int,LastName varchar(255),FirstName varchar(255),Address varchar(255),City varchar(255));'

In [17]:
cursor = connection.cursor()
cursor.execute(mySql_insert_query)
connection.commit()